In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('/home/users/akshay/PCPpred/MDCK/data/CycPeptMPDB_Peptide_Assay_MDCK (1).csv')
df.shape

(64, 247)

In [4]:
last_id = df['ID'].max()
print('last_id: ', last_id)

last_id:  8346


In [5]:
#Step 1: Removing unreliable datapoints (datapoints with Permeability value <= -10) and selecting the required Columns
df = df[df['Permeability'] > -10]
selected_columns = ['ID', 'SMILES', 'Permeability', 'Sequence', 'MolWt']
new_df = df[selected_columns]
new_df.shape

(64, 5)

In [6]:
new_df.head(10)

,ID,SMILES,Permeability,Sequence,MolWt
0,1017,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[C...,-6.55,"['L', 'dL', 'L', 'L', 'dP', 'Y']",712.933
1,1018,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)N(C)C(=O...,-4.50,"['L', 'Me_dL', 'meL', 'L', 'dP', 'meY']",755.014
2,1107,CC(C)C[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@@H](C)N...,-4.61,"['L', 'meL', 'L', 'Me_dA', 'meA', 'L', 'meL', ...",1047.438
3,1108,CC(C)C[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@@H](C)N...,-5.18,"['L', 'meA', 'L', 'Me_dA', 'meA', 'L', 'meA', ...",963.276
4,1109,C[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@@H](C)N(C)C(...,-6.12,"['A', 'meA', 'A', 'Me_dA', 'meA', 'A', 'meA', ...",794.952
5,1110,CC(C)C[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[C@H]...,-4.73,"['L', 'meL', 'L', 'Sar', 'P', 'L', 'meL', 'L',...",1043.406
6,1111,CC(C)C[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@H](CC(C...,-6.22,"['L', 'meA', 'L', 'Sar', 'P', 'L', 'meA', 'L',...",959.244
7,1112,C[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@H](C)NC(=O)[...,-6.17,"['A', 'meA', 'A', 'Sar', 'P', 'A', 'meA', 'A',...",790.920
8,1113,CC(C)C[C@@H]1NC(=O)[C@@H]2CCCN2C(=O)[C@@H](Cc2...,-5.82,"['L', 'meA', 'L', 'Me_dF', 'P', 'L', 'meA', 'L...",1139.494
9,1114,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=O)[C...,-4.94,"['L', 'meA', 'L', 'dP', 'meF', 'L', 'meA', 'L'...",1139.494


In [7]:
#Step 2: curating duplicate dapaoints based on SMILES representation
duplicates = new_df[new_df.duplicated(subset='SMILES', keep=False)]
non_duplicates = new_df[~new_df.duplicated(subset='SMILES', keep=False)]
print("DataFrame with duplicates:")
print(duplicates)
print("\nDataFrame without duplicates:")
print(non_duplicates)
duplicates.to_csv('/home/users/akshay/PCPpred/MDCK/data/duplicates_MDCK.csv', index=False)

DataFrame with duplicates:
Empty DataFrame
Columns: [ID, SMILES, Permeability, Sequence, MolWt]
Index: []

DataFrame without duplicates:
      ID                                             SMILES  Permeability  \
0   1017  CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[C...     -6.550000   
1   1018  CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)N(C)C(=O...     -4.500000   
2   1107  CC(C)C[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@@H](C)N...     -4.610000   
3   1108  CC(C)C[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@@H](C)N...     -5.180000   
4   1109  C[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@@H](C)N(C)C(...     -6.120000   
..   ...                                                ...           ...   
59  8342  CCOC(=O)[C@@H]1CSCC(=O)N[C@H](CC)C(=O)N(CC(C)C...     -4.994304   
60  8343  CCOC(=O)[C@@H]1CSCC(=O)N[C@H](CC)C(=O)N(CC(C)C...     -6.131653   
61  8344  CCOC(=O)[C@@H]1CSCC(=O)N[C@H](CC)C(=O)N(CC(C)C...     -5.241518   
62  8345  CCOC(=O)[C@@H]1CSCC(=O)N[C@@H](CC)C(=O)N(CC(C)...     -6.342777   
63  8346  CCOC(=

In [8]:
duplicates = new_df[new_df.duplicated(subset='SMILES', keep=False)]

non_duplicates = new_df[~new_df.duplicated(subset='SMILES', keep=False)]
#Permeability and MolWt of duplicate SMILES were averaged, and Most abundandant sequence was chosen
grouped_duplicates = duplicates.groupby('SMILES').agg(
    Permeability=('Permeability', 'mean'),   
    MolWt=('MolWt', 'mean'),   
    Sequence=('Sequence', lambda x: x.mode()[0]), 
).reset_index()

#Unique ID's were given to these curated datapoints, where ID's start from the last documented id in CycPeptMPDB_Peptide_Assay_PAMPA (5)
grouped_duplicates['ID'] = range(last_id + 1, last_id + 1 + len(grouped_duplicates))

grouped_duplicates = grouped_duplicates[['ID', 'SMILES', 'Permeability', 'Sequence', 'MolWt']]

non_duplicates_ordered = non_duplicates[['ID', 'SMILES', 'Permeability', 'Sequence', 'MolWt']]
merged_df = pd.concat([non_duplicates_ordered, grouped_duplicates], ignore_index=True)

print("DataFrame with processed duplicates:")
print(grouped_duplicates)

print("\nDataFrame without duplicates:")
print(non_duplicates)

print("Merged DataFrame:")
print(merged_df)

grouped_duplicates.to_csv('/home/users/akshay/PCPpred/MDCK/data/grouped_duplicates_MDCK.csv', index=False)
non_duplicates.to_csv('/home/users/akshay/PCPpred/MDCK/data/non_duplicates_MDCK.csv', index=False)
merged_df.to_csv('/home/users/akshay/PCPpred/MDCK/data/CycPeptMPDB_data_Main_MDCK.csv', index=False)

DataFrame with processed duplicates:
Empty DataFrame
Columns: [ID, SMILES, Permeability, Sequence, MolWt]
Index: []

DataFrame without duplicates:
      ID                                             SMILES  Permeability  \
0   1017  CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[C...     -6.550000   
1   1018  CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)N(C)C(=O...     -4.500000   
2   1107  CC(C)C[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@@H](C)N...     -4.610000   
3   1108  CC(C)C[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@@H](C)N...     -5.180000   
4   1109  C[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@@H](C)N(C)C(...     -6.120000   
..   ...                                                ...           ...   
59  8342  CCOC(=O)[C@@H]1CSCC(=O)N[C@H](CC)C(=O)N(CC(C)C...     -4.994304   
60  8343  CCOC(=O)[C@@H]1CSCC(=O)N[C@H](CC)C(=O)N(CC(C)C...     -6.131653   
61  8344  CCOC(=O)[C@@H]1CSCC(=O)N[C@H](CC)C(=O)N(CC(C)C...     -5.241518   
62  8345  CCOC(=O)[C@@H]1CSCC(=O)N[C@@H](CC)C(=O)N(CC(C)...     -6.342777   
63  83

In [9]:
#Step 3: Splitting the Main dataset into Train and Test in ration 0.8:0.2
df = pd.read_csv('/home/users/akshay/PCPpred/MDCK/data/CycPeptMPDB_data_Main_MDCK.csv')
df

,ID,SMILES,Permeability,Sequence,MolWt
0,1017,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[C...,-6.550000,"['L', 'dL', 'L', 'L', 'dP', 'Y']",712.933
1,1018,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)N(C)C(=O...,-4.500000,"['L', 'Me_dL', 'meL', 'L', 'dP', 'meY']",755.014
2,1107,CC(C)C[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@@H](C)N...,-4.610000,"['L', 'meL', 'L', 'Me_dA', 'meA', 'L', 'meL', ...",1047.438
3,1108,CC(C)C[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@@H](C)N...,-5.180000,"['L', 'meA', 'L', 'Me_dA', 'meA', 'L', 'meA', ...",963.276
4,1109,C[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@@H](C)N(C)C(...,-6.120000,"['A', 'meA', 'A', 'Me_dA', 'meA', 'A', 'meA', ...",794.952
...,...,...,...,...,...
59,8342,CCOC(=O)[C@@H]1CSCC(=O)N[C@H](CC)C(=O)N(CC(C)C...,-4.994304,"['dAbu', 'iBu_Gly', 'L', 'Me_dA', 'meA', 'dL',...",996.327
60,8343,CCOC(=O)[C@@H]1CSCC(=O)N[C@H](CC)C(=O)N(CC(C)C...,-6.131653,"['dAbu', 'iBu_Gly', 'L', 'Me_dA', 'meA', 'L', ...",996.327
61,8344,CCOC(=O)[C@@H]1CSCC(=O)N[C@H](CC)C(=O)N(CC(C)C...,-5.241518,"['dAbu', 'iBu_Gly', 'L', 'Me_dA', 'meA', 'dL',...",996.327
62,8345,CCOC(=O)[C@@H]1CSCC(=O)N[C@@H](CC)C(=O)N(CC(C)...,-6.342777,"['Abu', 'iBu_Gly', 'L', 'Me_dA', 'meA', 'dL', ...",996.327


In [10]:
#Main dataset is sorted based on Molecular weight in descending order
df_main = df.sort_values(by='MolWt', ascending= False)
df_main

,ID,SMILES,Permeability,Sequence,MolWt
15,1120,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=O)[C...,-6.30,"['L', 'meA', 'L', 'Me_dL', 'meF', 'L', 'meA', ...",1199.634
9,1114,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=O)[C...,-4.94,"['L', 'meA', 'L', 'dP', 'meF', 'L', 'meA', 'L'...",1139.494
8,1113,CC(C)C[C@@H]1NC(=O)[C@@H]2CCCN2C(=O)[C@@H](Cc2...,-5.82,"['L', 'meA', 'L', 'Me_dF', 'P', 'L', 'meA', 'L...",1139.494
12,1117,CC(C)C[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@H]2CCCN...,-5.65,"['L', 'meF', 'L', 'dP', 'meA', 'L', 'meF', 'L'...",1139.494
14,1119,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=O)[C...,-6.25,"['L', 'meA', 'L', 'Me_dA', 'meF', 'L', 'meA', ...",1115.472
...,...,...,...,...,...
0,1017,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[C...,-6.55,"['L', 'dL', 'L', 'L', 'dP', 'Y']",712.933
19,1842,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)NC(=O)[C@H...,-4.40,"['L', 'F', 'P', 'L', 'L', 'I']",696.934
17,1840,CC(C)C[C@@H]1NC(=O)[C@H](CC(C)C)NC(=O)[C@@H](C...,-4.48,"['L', 'dF', 'P', 'L', 'dL', 'L']",696.934
21,1844,CC(C)C[C@@H]1NC(=O)[C@H](CC(C)C)NC(=O)[C@@H](C...,-5.10,"['L', 'dF', 'dP', 'L', 'dL', 'L']",696.934


In [11]:
df_main['MolWt']

15    1199.634
9     1139.494
8     1139.494
12    1139.494
14    1115.472
        ...   
0      712.933
19     696.934
17     696.934
21     696.934
23     696.934
Name: MolWt, Length: 64, dtype: float64

In [12]:
#Every fifth datapoint is allocated to test set and remaining datapoints to Train set
test_mask = np.arange(len(df_main)) % 5 == 0
test_df = df_main[test_mask]
test_df.shape

(13, 5)

In [13]:
train_df = df_main[~test_mask]
train_df.shape

(51, 5)

In [14]:
print(train_df["Permeability"].max())
print(train_df["Permeability"].min())
print(train_df.shape)

-4.41
-7.82
(51, 5)


In [15]:
print(test_df["Permeability"].max())
print(test_df["Permeability"].min())
print(test_df.shape)

-4.4
-7.21
(13, 5)


In [16]:
test_df.to_csv("/home/users/akshay/PCPpred/MDCK/data/Test_MDCK.csv",index=False)
train_df.to_csv("/home/users/akshay/PCPpred/MDCK/data/Train_MDCK.csv",index=False)

In [17]:
train_df

,ID,SMILES,Permeability,Sequence,MolWt
9,1114,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=O)[C...,-4.940000,"['L', 'meA', 'L', 'dP', 'meF', 'L', 'meA', 'L'...",1139.494
8,1113,CC(C)C[C@@H]1NC(=O)[C@@H]2CCCN2C(=O)[C@@H](Cc2...,-5.820000,"['L', 'meA', 'L', 'Me_dF', 'P', 'L', 'meA', 'L...",1139.494
12,1117,CC(C)C[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@H]2CCCN...,-5.650000,"['L', 'meF', 'L', 'dP', 'meA', 'L', 'meF', 'L'...",1139.494
14,1119,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=O)[C...,-6.250000,"['L', 'meA', 'L', 'Me_dA', 'meF', 'L', 'meA', ...",1115.472
27,2428,CCN1CC(=O)N[C@@H](CC(C)C)C(=O)N(C)[C@@H](C)C(=...,-5.350000,"['L', 'dP', 'Mono38', 'L', 'meA', 'F', 'dP', '...",1091.406
30,2446,CCN1CC(=O)N[C@@H](CC(C)C)C(=O)N2CCC[C@@H]2C(=O...,-6.850000,"['L', 'dP', 'meL', 'A', 'Mono38', 'F', 'dP', '...",1091.406
29,2445,CCN1CC(=O)N[C@@H](CC(C)C)C(=O)N2CCC[C@@H]2C(=O...,-5.270000,"['L', 'dP', 'meL', 'A', 'cHexCH2_Gly', 'F', 'd...",1089.434
26,2427,CCN1CC(=O)N[C@@H](CC(C)C)C(=O)N(C)[C@@H](C)C(=...,-6.340000,"['L', 'dP', 'cHexCH2_Gly', 'L', 'meA', 'F', 'd...",1089.434
51,8145,CCC[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@H](CC(C)C)...,-6.569578,"['L', 'meA', 'Nva', 'dP', 'Me_Abu', 'Cha', 'me...",1048.403
2,1107,CC(C)C[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@@H](C)N...,-4.610000,"['L', 'meL', 'L', 'Me_dA', 'meA', 'L', 'meL', ...",1047.438


In [18]:
test_df

,ID,SMILES,Permeability,Sequence,MolWt
15,1120,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=O)[C...,-6.300000,"['L', 'meA', 'L', 'Me_dL', 'meF', 'L', 'meA', ...",1199.634
13,1118,CC(C)C[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@@H](Cc2...,-5.350000,"['L', 'meA', 'L', 'Me_dF', 'meA', 'L', 'meA', ...",1115.472
16,1121,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)NC(=O)[C@H]...,-6.200000,"['L', 'A', 'L', 'dP', 'F', 'L', 'A', 'L', 'dP'...",1083.386
47,8133,CCC[C@@H]1NC(=O)CN(CC)C(=O)[C@H](CC(C)C)NC(=O)...,-5.965681,"['L', 'Et_Gly', 'Nva', 'dP', 'meA', 'Cha', 'me...",1034.376
50,8143,CCC[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@H](CC(C)C)...,-5.676306,"['L', 'meA', 'Nva', 'dP', 'Me_Nva', 'L', 'meA'...",1022.365
42,8119,CCC[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@H](CC(C)C)...,-5.516110,"['L', 'meA', 'Nva', 'dP', 'Pr_Gly', 'L', 'meA'...",1008.338
35,6496,CC(=O)N1CCC[C@H]1C(=O)N(C)[C@H](CC(C)C)C(=O)N[...,-5.060000,"['ac-', 'P', 'Me_dL', 'T', 'dL', 'L', 'dL', 'M...",1002.309
56,8168,CCOC(=O)[C@@H]1CSCC(=O)N[C@@H](CC)C(=O)N(C)[C@...,-5.977700,"['Abu', 'meA', 'L', 'Me_dA', 'iBu_Gly', 'dL', ...",996.327
62,8345,CCOC(=O)[C@@H]1CSCC(=O)N[C@@H](CC)C(=O)N(CC(C)...,-6.342777,"['Abu', 'iBu_Gly', 'L', 'Me_dA', 'meA', 'dL', ...",996.327
34,6423,CC(=O)N1CCC[C@@H]1C(=O)N(C)[C@@H](CC(C)C)C(=O)...,-7.210000,"['ac-', 'dP', 'meL', 'T', 'dL', 'dL', 'L', 'A'...",988.282


In [19]:
import pandas as pd

df = pd.read_csv('/home/users/akshay/PCPpred/MDCK/data/Train_MDCK.csv')

with open('/home/users/akshay/PCPpred/MDCK/data/Train_MDCK.smi', 'w') as f:
    for _, row in df.iterrows():
        smiles = row['SMILES']
        id = row['ID']
        f.write(f"{smiles} {id}\n")
print("train.smi file has been created successfully.")

train.smi file has been created successfully.


In [20]:
df = pd.read_csv('/home/users/akshay/PCPpred/MDCK/data/Test_MDCK.csv')

with open('/home/users/akshay/PCPpred/MDCK/data/Test_MDCK.smi', 'w') as f:
    for _, row in df.iterrows():
        smiles = row['SMILES']
        id = row['ID']
        f.write(f"{smiles} {id}\n")
print("test.smi file has been created successfully.")

test.smi file has been created successfully.
